## Homework 2 Solution Template
### CSCI 4270 / 6270
### Due: February 5, 2024

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

## Problem 1

In [2]:
def print_matrix(name, A):
    print(f'Matrix {name}:')
    for i in range(A.shape[0]):
        row = A[i, :]
        print(f'{row[0]:.1f}', end='')
        for j in range(1, row.shape[0]):
            print(f', {row[j]:.1f}', end='')
        print()

In [4]:
"""
Problem 1, Test 1. This is a very simple test that could be easily checked by hand.
"""
params = [
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 1000, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

p1_camera(params, points)


Matrix R:
1.0, 0.0, 0.0
0.0, 1.0, 0.0
0.0, 0.0, 1.0
Matrix K:
1500.0, 0.0, 2995.0
0.0, 1500.0, 2001.0
0.0, 0.0, 1.0
Matrix M:
1500.0, 0.0, 2995.0, -29950.0
0.0, 1500.0, 2001.0, -20010.0
0.0, 0.0, 1.0, -10.0
Projections:
0: point 10.0 5.0 100.0 ==> 2084.3, 3161.7, inside
1: point 0.0 0.0 0.5 ==> 2001.0, 2995.0, inside
2: point -30.0 10.0 -20.0 ==> 1501.0, 4495.0, inside
3: point 20.0 15.0 20.0 ==> 4251.0, 5995.0, outside
visible: 0 3
hidden: 1 2


In [5]:
'''
Problem 1, Test 2
'''
params = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)


Matrix R:
0.7, -0.1, -0.7
-0.0, 1.0, -0.2
0.7, 0.1, 0.7
Matrix K:
1000.0, 0.0, 3005.0
0.0, 1000.0, 1998.0
0.0, 0.0, 1.0
Matrix M:
-1428.5, -562.5, 2770.0, -5112.7
-1535.6, 617.4, 1500.9, -27388.2
-0.7, -0.2, 0.7, 1.5
Projections:
0: point 100.0 15.0 90.0 ==> 3487.2, -8851.4, outside
1: point -100.0 800.0 1500.0 ==> 3021.6, 4043.8, inside
2: point 10.0 -500.0 -500.0 ==> 4311.3, 4394.6, outside
3: point -30.0 10.0 20.0 ==> 1589.0, 2534.4, inside
visible: 1 3
hidden: 0 2


In [ ]:
'''
Problem 1, Test 3
'''
params = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)

## Problem 2

In [7]:
def load_points(fn):
    '''
    Input: a path to a file containing x, y points, one point per line.
    Returns: two-d np array where each row contains an x, y point
    '''
    f = open(fn, 'r')
    pts = []
    for line in f:
        line = line.strip().split()
        x, y = float(line[0]), float(line[1])
        pts.append([x, y])
    pts = np.array(pts)
    f.close()
    return pts

In [8]:
def p2_ransac(fn, samples, tau, seed):


In [9]:
'''
Problem 2, Test 1
'''
fn = 'data/p2_pts1_in.txt'
samples = 25
tau = 2.5
seed = 999
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (0,28)
line (-0.983,0.184,-26.286)
inliers 13

Sample 3:
indices (27,25)
line (0.426,0.905,-4.913)
inliers 19

Sample 10:
indices (23,4)
line (0.545,0.838,-0.944)
inliers 21

avg inlier dist 0.739
avg outlier dist 8.920


In [ ]:
'''
Problem 2, Test 2
'''
fn = 'data/p2_pts2_in.txt'
samples = 35
tau = 3.0
seed = 1232
p2_ransac(fn, samples, tau, seed)


## Problem 3 (4270 Only)


In [3]:
'''
Utility for Problem 3
'''
import os
from os import sys

def get_images(img_dir: str) -> tuple[list[str], list[np.ndarray]]:
    start_cwd:str = os.getcwd()
    os.chdir(img_dir)
    img_name_list: list[str] = os.listdir('./')
    img_name_list = [name for name in img_name_list if 'jpg' in name.lower()]
    img_name_list.sort()

    img_list: list[np.ndarray] = []
    im: np.ndarray

    for i_name in img_name_list:
        im = cv2.imread(i_name, cv2.IMREAD_GRAYSCALE)
        if im is None:
            print('Could not open', i_name)
            sys.exit(0)
        img_list.append(im)

    os.chdir(start_cwd)
    return img_name_list, img_list

In [24]:
def calculate_image_energy(image: np.ndarray) -> float:
  """
  Using Gaussian Smooting, we find the energy value of the image.

  Args:
    image (np.ndarray): Image to be evaluated

  Returns:
    float -> image's energy
  """

  # Compute derivatives and apply aspects of Gaussian Smooting to images w.r.t x and y axes
  im_xdelta: np.ndarray = cv2.Sobel(image, cv2.CV_32F, 1, 0) # Gradients or differences of all pxels across the x-axis
  im_ydelta: np.ndarray =  cv2.Sobel(image, cv2.CV_32F, 0, 1) # Gradients or differences of all pxels across the y-axis
  M, N = image.shape

  gradient_magnitude_squared: np.ndarray = ((im_xdelta) ** 2 + (im_ydelta) ** 2)
  total_gradient_magnitude_squared: np.ndarray = np.sum(gradient_magnitude_squared) # Sums all gradients within the image
  normalized_total: np.ndarray = (1 / (M * N)) * total_gradient_magnitude_squared

  return normalized_total

def find_best_focused_image(images: list[np.ndarray]) -> tuple[np.ndarray, int, list[float]]:
  """
  Using Gaussian Smooting, we find the energy value of each image in {images} and declare the best focused image as the image with the highest energy value.

  Args:
    images (list[np.ndarray]): List of images to be evaluated

  Returns:
    tuple[
      np.ndarray -> image with the best focus,
      int -> index of image with the best focus w.r.t the list it is in
  """
  image_energies: list[np.ndarray] = [] # Entry 1 corresponds to Entry 1 in images
  E_image: float
  for image in images:
    E_image = calculate_image_energy(image=image)
    image_energies.append(E_image)

  best_focused_image_index: int = np.argmax(image_energies)
  best_focused_image: np.ndarray = image[best_focused_image_index]
  return best_focused_image,best_focused_image_index,image_energies

In [25]:
evergreen_dir = 'data/evergreen'
names_of_images, images = get_images(img_dir=evergreen_dir)
best_focused_image, best_focused_image_index, image_energies = find_best_focused_image(images=images)

E_image: np.ndarray
for i, image_name in enumerate(names_of_images):
  E_image = image_energies[i]
  print(f"{image_name}: {E_image:.1f}")
print(f"Image {names_of_images[best_focused_image_index]} is best focused.")

DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


In [26]:
def p3_best_focus(image_dir: str) -> tuple[np.ndarray, int]:
  """
  Finds and outputs best foucused image

  Args:
    image_dir (str): Directory where images can be found

  Returns:
      tuple[
        np.ndarray -> image with the best focus,
        int -> index of image with the best focus -- corresponding to the index - 1 of the the sorted images in {image_dir}
  """
  names_of_images, images = get_images(img_dir=image_dir)
  best_focused_image, best_focused_image_index, image_energies = find_best_focused_image(images=images)

  E_image: np.ndarray
  for i, image_name in enumerate(names_of_images):
    E_image = image_energies[i]
    print(f"{image_name}: {E_image:.1f}")
  print(f"Image {names_of_images[best_focused_image_index]} is best focused.")
  return best_focused_image, best_focused_image_index


In [27]:
evergreen_dir = 'data/evergreen'
p3_best_focus(evergreen_dir)

DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


(array([146, 148, 148, 144, 145, 151, 151, 154, 160, 169, 156, 155, 160,
        162, 167, 164, 164, 160, 165, 163, 165, 168, 163, 153, 148, 141,
        140, 139, 141, 138, 136, 135, 134, 126, 127, 145, 140, 151, 202,
        229, 231, 225, 226, 231, 230, 222, 226, 226, 239, 249, 237, 231,
        229, 234, 239, 235, 158, 159, 162, 166, 170, 173, 176, 177, 179,
        180, 181, 181, 181, 180, 180, 179, 181, 181, 181, 181, 180, 181,
        179, 173, 171, 171, 169, 168, 168, 174, 232, 237, 235, 239, 244,
        239, 188, 153, 163, 163, 168, 171, 175, 178, 179, 180, 180, 179,
        179, 179, 179, 178, 177, 177, 176, 175, 174, 174, 173, 176, 172,
        168, 167, 163, 165, 164, 164, 191, 232, 232, 197, 219, 221, 222,
        226, 228, 231, 232, 229, 232, 234, 235, 243, 246, 247, 248, 247,
        248, 250, 249, 241, 239, 200, 142, 148, 138, 141, 151, 140, 138,
        133, 139, 154, 155, 158, 153, 148, 141, 142, 124, 114, 109,  76,
        114, 129, 120, 122, 120, 127, 103, 146, 134

In [13]:
image_dir = 'data/evergreen'
p3_best_focus(image_dir)

DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


In [ ]:
image_dir = 'data/branches'
p3_best_focus(image_dir)